In [1]:
import pandas as pd
# from pandarallel import pandarallel
import numpy as np
import seaborn as sns
import plotly.express as px
import torch
import os
from tqdm import tqdm
import pickle
from Levenshtein import distance as lev_distance

import warnings

warnings.filterwarnings('ignore')
sns.set(rc={'figure.figsize': (20, 10), 'figure.facecolor': 'white'})
sns.set_palette("viridis")
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pandarallel.initialize(progress_bar=True)
os.environ["TOKENIZERS_PARALLELISM"] = "true"  # activate parallelism

# Stage 1 - Get and preprocess Data

In [2]:
def join_non_null_values(l):
    res = [str(x) for x in l if str(x) != 'nan']
    return ', '.join(res)

In [123]:
train = (
    pd.read_csv('../data/raw/additional_data/building_20230808.csv').add_suffix('_building')
    .merge(
        pd.read_csv('../data/raw/additional_data/district_20230808.csv').add_suffix('_district'),
        left_on='district_id_building',
        right_on='id_district',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/prefix_20230808.csv').add_suffix('_prefix'),
        left_on='prefix_id_building',
        right_on='id_prefix',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/town_20230808.csv').add_suffix('_town'),
        left_on='town_id_prefix',
        right_on='id_town',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/geonim_20230808.csv').add_suffix('_geonim'),
        left_on='geonim_id_prefix',
        right_on='id_geonim',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/geonimtype_20230808.csv').add_suffix('_geonimtype'),
        left_on='type_id_geonim',
        right_on='id_geonimtype',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/area_20230808.csv').add_suffix('_area'),
        left_on='area_id_prefix',
        right_on='id_area',
        how='left',
    )
    .merge(
        pd.read_csv('../data/raw/additional_data/areatype_20230808.csv').add_suffix('_areatype'),
        left_on='type_id_area',
        right_on='id_areatype',
        how='left',
    )
    .assign(name_district_full=lambda df_: df_.name_district + ' район')
    .assign(
        all_in_field=lambda df_: df_[['full_address_building', 'type_building',
                                      'name_district_full', 'name_area', 'name_areatype']]
        .apply(lambda x: join_non_null_values(x), axis=1)
    )
)
train.to_pickle("../data/processed/main_data.pkl")
train

,id_building,prefix_id_building,district_id_building,house_building,corpus_building,liter_building,villa_building,parcel_building,full_address_building,is_updated_building,is_actual_building,type_building,municipality_id_building,short_address_building,post_prefix_building,build_number_building,id_district,name_district,is_updated_district,is_actual_district,id_prefix,town_id_prefix,geonim_id_prefix,area_id_prefix,toponim_id_prefix,name_prefix,short_name_prefix,search_index_prefix,is_updated_prefix,is_actual_prefix,sub_rf_id_prefix,has_buildings_prefix,id_town,name_town,short_name_town,search_index_town,is_updated_town,is_actual_town,has_buildings_town,id_geonim,type_id_geonim,name_geonim,short_name_geonim,is_updated_geonim,is_actual_geonim,only_name_geonim,id_geonimtype,name_geonimtype,short_name_geonimtype,is_updated_geonimtype,is_actual_geonimtype,id_area,type_id_area,name_area,short_name_area,is_updated_area,is_actual_area,only_name_area,id_areatype,name_areatype,short_name_areatype,is_updated_areatype,is_actual_areatype,name_district_full,all_in_field
0,56343,11132,35,12,NaN,А,NaN,NaN,"город Пушкин, Кедринская улица, дом 12",True,False,NaN,107.0,"г.Пушкин, Кедринская ул., д. 12",NaN,NaN,35,Пушкинский,True,True,11132,28.0,4801.0,NaN,NaN,"город Пушкин, Кедринская улица","г.Пушкин, Кедринская ул.","'город':1,5 'кедринск':7 'кедринская':3 'пушкин':2,6 'улиц':8 'улица':4",True,True,15.0,True,28.0,город Пушкин,г.Пушкин,"'город':1,3 'пушкин':2,4",True,True,True,4801.0,12.0,Кедринская улица,Кедринская ул.,True,True,Кедринская,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Пушкинский район,"город Пушкин, Кедринская улица, дом 12, Пушкинский район"
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,"поселок Ушково, Пляжевая улица, дом 4Б",True,False,NaN,128.0,"пос. Ушково, Пляжевая ул., д. 4Б",NaN,NaN,38,Курортный,True,True,6987,46.0,3.0,NaN,NaN,"посёлок Ушково, Пляжевая улица","пос. Ушково, Пляжевая ул.",'пляжев':7 'пляжевая':3 'поселок':5 'посёлок':1 'улиц':8 'улица':4 'ушков':6 'ушково':2,True,True,15.0,True,46.0,посёлок Ушково,пос. Ушково,'поселок':3 'посёлок':1 'ушков':4 'ушково':2,True,True,True,3.0,12.0,Пляжевая улица,Пляжевая ул.,True,True,Пляжевая,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Курортный район,"поселок Ушково, Пляжевая улица, дом 4Б, Курортный район"
2,7134,6469,15,30,2,Е,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е",True,False,Нежилое,30.0,"г.Санкт-Петербург, пр. Маршала Жукова, д. 30, к. 2, л. Е",198303,NaN,15,Кировский,True,True,6469,36.0,2194.0,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова","г.Санкт-Петербург, пр. Маршала Жукова","'г':1,8 'жуков':14 'жукова':7 'марша':13 'маршала':6 'петербург':4,11 'проспект':5,12 'санкт':3,10 'санкт-петербург':2,9",True,True,15.0,True,36.0,г.Санкт-Петербург,г.Санкт-Петербург,"'г':1,5 'петербург':4,8 'санкт':3,7 'санкт-петербург':2,6",True,True,True,2194.0,10.0,проспект Маршала Жукова,пр. Маршала Жукова,True,True,Маршала Жукова,10.0,проспект,пр.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Кировский район,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е, Нежилое, Кировский район"
3,124415,7838,38,5,2,А,NaN,NaN,"поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2",True,False,Нежилое,110.0,"пос. Белоостров, Дюны, Центральная ул., д. 5, к. 2",NaN,NaN,38,Курортный,True,True,7838,37.0,319.0,NaN,161.0,"посёлок Белоостров, Дюны, Центральная улица","пос. Белоостров, Дюны, Центральная ул.",'белоостр':7 'белоостров':2 'дюн':8 'дюны':3 'поселок':6 'посёлок':1 'улиц':10 'улица':5 'центральн':9 'центральная':4,True,True,15.0,True,37.0,посёлок Белоостров,пос. Белоостров,'белоостр':4 'белоостров':2 'поселок':3 'посёлок':1,True,True,True,319.0,12.0,Центральная улица,Центральная ул.,True,True,Центральная,12.0,улица,ул.,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Курортный район,"поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2, Нежилое, Курор

In [8]:
for_embeddings = (
    pd.DataFrame(
        np.concatenate([
            train[['id_building', 'full_address_building']].values,
            train[['id_building', 'all_in_field']].values,
            # train[['id_building', 'short_address_building']].values,
            # train[['id_building', 'name_prefix']].values,
            # train[['id_building', 'short_name_prefix']].values,
            # train[['id_building', 'name_town']].values,
            # train[['id_building', 'short_name_town']].values,
            # train[['id_building', 'name_geonim']].values,
            # train[['id_building', 'short_name_geonim']].values,
            # train[['id_building', 'only_name_geonim']].values,
            # train[['id_building', 'name_prefix']].values,
            # train[['id_building', 'short_name_prefix']].values,
            # train[['id_building', 'name_geonimtype']].values,
            # train[['id_building', 'only_name_area']].values,
            # train[['id_building', 'name_areatype']].values,
            # train[['id_building', 'short_name_areatype']].values,
            # train[['id_building', 'name_district']].values,
            # train[['id_building', 'name_district_full']].values,
        ], axis=0),
        columns=['id_building', 'address']
    )
    .dropna()
    .reset_index(drop=True)
)

In [10]:
for_embeddings.to_pickle('../data/processed/for_embeddings_with_names.pkl')
for_embeddings

,id_building,address
0,56343,"город Пушкин, Кедринская улица, дом 12"
1,595,"поселок Ушково, Пляжевая улица, дом 4Б"
2,7134,"г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е"
3,124415,"поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2"
4,185368,"поселок Песочный, Речная улица, дом 28, литера Б"
...,...,...
333285,18941,"г.Санкт-Петербург, Бородинская улица, дом 3, литера В, Нежилое, Адмиралтейский район"
333286,18942,"посёлок Александровская, 2-я линия, дом 34/3, литера А, Жилое, Пушкинский район"
333287,18943,"посёлок Ушково, Детская улица, дом 39, Курортный район"
333288,18944,"г.Санкт-Петербург, Заусадебная улица, дом 25, литера Б, Нежилое, Приморский район"


# Step 2 - Get Embeddings

In [124]:
from sentence_transformers import SentenceTransformer #, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# embeddings_list = [0] * len(for_embeddings)

In [12]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [13]:
# embeddings = tokenizer(list(for_embeddings['address']), padding=True, truncation=True, return_tensors='pt')

In [126]:
embeddings = model.encode(for_embeddings['address'].astype('str'), convert_to_tensor=True, show_progress_bar=True,
                          batch_size=128)

In [24]:
with open('../data/processed/embeddings.pkl', 'wb') as fp:
    pickle.dump(embeddings, fp)

In [25]:
# embeddings.to_pickle("../data/processed/embeddings.pkl")

In [160]:
# #split DataFrame into chunks and iter over it creating embeddings
# n = 1000 #specify number of rows in each chunk
# embeddings = torch.empty(0)
# for batch in tqdm([for_embeddings[i:i + n] for i in range(0, len(for_embeddings), n)]):
#     batch_embedding = model.encode(batch['address'].astype('str'), convert_to_tensor=True, show_progress_bar=True)
#     embeddings = torch.concat([embeddings, batch_embedding])

In [161]:
# batch

In [ ]:
# for_embeddings['address'].parallel_apply(lambda x: model.encode(x, convert_to_tensor=True))

In [162]:

# embeddings_list = []
# for indx in tqdm(range(len(for_embeddings))):
# # train_emb_only.loc[indx, 'full_address_embedding'] = Fake(model.encode(train_emb_only.loc[indx, 'full_address'], convert_to_tensor=True))
#     embeddings_list.append(model.encode(for_embeddings.loc[indx, 'address'], convert_to_tensor=True))
# # train_emb_only.full_address_embedding = train_emb_only.full_address_embedding.apply(lambda x: x.obj)
# embeddings_list

# Step 3 - Similarity Matcher

In [81]:
QUERY = ["г.Санкт-Петербург, набережная Обводного канала, дом 205, литера М"]

In [ ]:
with open('../data/processed/embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

for_embeddings = pd.read_pickle("../data/processed/for_embeddings_with_names.pkl")

In [127]:
def get_best_matches(query, top_n, embeddings: np.array, sentences):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    query_embedding = model.encode(query, convert_to_tensor=True)
    scores = embeddings.dot(query_embedding.T).ravel()
    best = np.argpartition(scores, -top_n)[-top_n:]
    sentences['scores'] = scores
    sentences.loc[list(best), 'lev_distance'] = sentences.loc[list(best), 'address'].apply(
        lambda x: lev_distance(query, x))
    best_matches = sentences[['id_building', 'address', 'scores', 'lev_distance']].loc[list(best)]
    best_matches_within_id_indices = best_matches.groupby(['id_building']).scores.transform(max) == best_matches.scores
    best_matches = best_matches.loc[best_matches_within_id_indices].sort_values(by='scores',
                                                                                ascending=False).reset_index(drop=True)
    if len(best_matches[best_matches['scores'] >= 0.98]):
        return best_matches[best_matches['scores'] >= 0.98]
    elif len(best_matches[best_matches['scores'] >= 0.90]):
        return best_matches[best_matches['scores'] >= 0.90]
    else:
        return best_matches
    # return best_matches.loc[best_matches_within_id_indices].sort_values(by='lev_distance').reset_index(drop=True)

In [128]:
get_best_matches(QUERY, 10, np.array(embeddings), for_embeddings)

,id_building,address,scores,lev_distance
0,84908,"г.Санкт-Петербург, набережная Обводного канала, дом 205, литера М",1.000000,65.0
1,31542,"г.Санкт-Петербург, набережная Обводного канала, дом 205, литера А",0.990537,65.0


In [119]:
def slow_get_best_matches(query, top_n, embeddings: np.array, sentences):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    query_embedding = model.encode(query, convert_to_tensor=True)
    scores = embeddings.dot(query_embedding.T).ravel()
    best = np.argpartition(scores, -top_n)[-top_n:]
    sentences['scores'] = scores
    sentences.loc[list(best), 'lev_distance'] = sentences.loc[list(best), 'address'].apply(
        lambda x: lev_distance(query, x))
    best_matches = sentences[['id_building', 'address', 'scores', 'lev_distance']].loc[list(best)]
    add_data = pd.read_pickle("../data/processed/main_data.pkl")
    best_matches = best_matches.merge(add_data[['id_building', 'liter_building', 'name_district_full', 'name_town']])
    best_matches_within_id_indices = best_matches.groupby(['id_building']).scores.transform(max) == best_matches.scores
    best_matches = best_matches.loc[best_matches_within_id_indices].sort_values(by='scores',
                                                                                ascending=False).reset_index(drop=True)

    # add logic to topn selection
    # add catboost here to ass probabilities
    if len(best_matches[best_matches['scores'] >= 0.98]):
        return best_matches[best_matches['scores'] >= 0.98]
    elif len(best_matches[best_matches['scores'] >= 0.90]):
        return best_matches[best_matches['scores'] >= 0.90]
    else:
        return best_matches

In [120]:
slow_get_best_matches(QUERY, 10, np.array(embeddings), for_embeddings)

,id_building,address,scores,lev_distance,liter_building,name_district_full,name_town
0,84908,"г.Санкт-Петербург, набережная Обводного канала, дом 205, литера М",1.000000,65.0,М,Адмиралтейский район,г.Санкт-Петербург
1,31542,"г.Санкт-Петербург, набережная Обводного канала, дом 205, литера А",0.990537,65.0,А,Адмиралтейский район,г.Санкт-Петербург


test:
г.Санкт-Петербург, дом 223-225, наб. Обводного канала
г.Санкт-Петербург, дом 223-225
посёлок Солнечное, Комсомольская улица